In [13]:
!ls

 GPT_generated_essays.csv	 generate.py
 GPT_generated_essays_1.csv	 generated_essays.csv
 Generators			 glove.6B.100d.txt
'LLM Detect AI Generated Text'	 glove.6B.200d.txt
 NoteBooks			 glove.6B.300d.txt
 Palm_generated_texts.csv	 glove.6B.50d.txt
 Palm_generated_texts1.csv	 glove.6B.zip
 Palm_generated_texts2.csv	 new_train_essay.csv
 README.md			 quick_start_pytorch_images
 concat_generated_df.csv	 requirements.txt


In [ ]:
!wget https://nlp.stanford.edu/data/wordvecs/glove.6B.zip

In [6]:
import zipfile

with zipfile.ZipFile('glove.6B.zip', 'r') as zip_ref:
    zip_ref.extractall('.')


 GPT_generated_essays.csv	 generate.py
 GPT_generated_essays_1.csv	 generated_essays.csv
 Generators			 glove.6B.100d.txt
'LLM Detect AI Generated Text'	 glove.6B.200d.txt
 NoteBooks			 glove.6B.300d.txt
 Palm_generated_texts.csv	 glove.6B.50d.txt
 Palm_generated_texts1.csv	 glove.6B.zip
 Palm_generated_texts2.csv	 new_train_essay.csv
 README.md			 quick_start_pytorch_images
 concat_generated_df.csv	 requirements.txt


In [8]:
import nltk
import spacy

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

!python -m spacy download en_core_web_sm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 62.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

# Function to preprocess text
def preprocess_text(text, use_tfidf=True, max_features=None):
    # Remove special characters and digits
    text = re.sub("(\\d|\\W)+", " ", text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenization
    tokens = nltk.word_tokenize(text)
    
    # Removing stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization with spaCy
    lemmatized_text = " ".join([token.lemma_ for token in nlp(" ".join(filtered_tokens))])
    
    # Optional: TF-IDF Vectorization
    if use_tfidf:
        vectorizer = TfidfVectorizer(max_features=max_features)
        lemmatized_text = vectorizer.fit_transform([lemmatized_text]).toarray()
    
    return lemmatized_text

# Example usage
text = "Example text to be preprocessed."
processed_text = preprocess_text(text)

array([[0.57735027, 0.57735027, 0.57735027]])

In [11]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')

def pos_features(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    pos_counts = nltk.FreqDist(tag for (word, tag) in pos_tags)
    return pos_counts

# Example usage
text = "Your example text here."
pos_tags = pos_features(text)
pos_tags

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


FreqDist({'NN': 2, 'PRP$': 1, 'RB': 1, '.': 1})

In [16]:
def ner_features(text):
    doc = nlp(text)
    entity_counts = {}
    for ent in doc.ents:
        entity_counts[ent.label_] = entity_counts.get(ent.label_, 0) + 1
    return entity_counts

# Example usage
text = "Your example text here."
entities = ner_features(text)
entities

{}

In [15]:
import numpy as np

def load_glove_model(glove_file):
    with open(glove_file, 'r', encoding='utf-8') as file:
        model = {}
        for line in file:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            model[word] = embedding
        return model

glove_model = load_glove_model("glove.6B.300d.txt")


In [17]:
def vectorize_text_with_glove(text, glove_model):
    tokens = word_tokenize(text.lower())
    token_embeddings = [glove_model[token] for token in tokens if token in glove_model]
    if not token_embeddings:
        return np.zeros(100)  # Size of GloVe vectors
    return np.mean(token_embeddings, axis=0)

# Example usage
text = "Your example text here."
vectorized_text = vectorize_text_with_glove(text, glove_model)

In [19]:
!ls

 GPT_generated_essays.csv	 generate.py
 GPT_generated_essays_1.csv	 generated_essays.csv
 Generators			 glove.6B.100d.txt
'LLM Detect AI Generated Text'	 glove.6B.200d.txt
 NoteBooks			 glove.6B.300d.txt
 Palm_generated_texts.csv	 glove.6B.50d.txt
 Palm_generated_texts1.csv	 glove.6B.zip
 Palm_generated_texts2.csv	 new_train_essay.csv
 README.md			 quick_start_pytorch_images
 concat_generated_df.csv	 requirements.txt


In [22]:
import pandas as pd
# my files
train_essays_df = pd.read_csv('new_train_essay.csv')
train_prompts_df = pd.read_csv('/datasets/essays-training/train_prompts.csv')

In [27]:
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

def preprocess_text(text, use_tfidf=True, max_features=None):
    # Remove special characters and digits    
    text = re.sub("(\\d|\\W)+", " ", text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization with spaCy
    lemmatized_text = " ".join([token.lemma_ for token in nlp(" ".join(filtered_tokens))])
    
    # Optional: TF-IDF Vectorization
    if use_tfidf:
        vectorizer = TfidfVectorizer(max_features=max_features)
        lemmatized_text = vectorizer.fit_transform([lemmatized_text]).toarray()
    
    return lemmatized_text

# Apply the preprocessing function to the DataFrame
train_essays_df['processed_text'] = train_essays_df['text'].apply(lambda x: preprocess_text(x))
# Apply the POS tagging function
train_essays_df['pos_tags'] = train_essays_df['processed_text'].apply(pos_features)

# Apply the NER function
train_essays_df['ner_tags'] = train_essays_df['processed_text'].apply(ner_features)
# Assuming vectorize_text_with_glove function and GloVe model are already defined

# Apply the word embedding function
train_essays_df['word_embeddings'] = train_essays_df['processed_text'].apply(lambda x: vectorize_text_with_glove(x, glove_model))
# Combine POS, NER, and word embedding features
combined_features = pd.concat([train_essays_df['pos_tags'], train_essays_df['ner_tags'], pd.DataFrame(train_essays_df['word_embeddings'].tolist())], axis=1)


TypeError: cannot use a string pattern on a bytes-like object

In [ ]:
# Final feature matrix and target variable
X = combined_features
y = train_essays_df['generated']  # or the appropriate target variable column
